# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [ ]:
%%capture
#!pip install wandb
!pip install pandas

### Step 1️⃣: Import W&B and Login

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from tensorflow.keras.callbacks import ReduceLROnPlateau


#%matplotlib inline

In [ ]:
import wandb
from wandb.keras import WandbCallback

#wandb.init()  # defaults are over-ridden during the sweep


#wandb.login()

In [ ]:

fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

Read data in

# 👩‍🍳 Prepare Dataset

In [ ]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


## Normalize data
as shown in https://keras.io/guides/preprocessing_layers/#normalizing-numerical-features 

In [ ]:

x_train_norm = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_train_norm

# Build simple Model

In [ ]:
# AoModel = keras.Sequential([
#      keras.layers.InputLayer(19, name="digits"),
#      keras.layers.Dense(2000, activation="relu"),
#      keras.layers.Dense(1050, activation="relu"),
#      keras.layers.Dense(100, activation="relu"),
#      keras.layers.Dense(9, activation="linear", name="predictions"),

# ])
# AoModel.summary()

In [ ]:
# batch_size = 128
# epochs = 200


#     # optimizer='sgd',
#     # loss='mse',
#     # metrics=[tf.keras.metrics.RootMeanSquaredError()])



# AoModel.compile(loss= keras.losses.MeanSquaredError(), optimizer="adam", metrics= [tf.keras.metrics.RootMeanSquaredError()])
# #AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,  callbacks=[WandbCallback()])

# #AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

# 🧠 Define the wandb sweep

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoKerasStudio.ipynb"

sweep_config = {
    "method": "random", # try grid or random
    "metric": {
        "name": "val_root_mean_squared_error",
        "goal": "minimize"
    },
    "parameters": {

    "learning_rate" :{
        "values": [ 0.005, 0.001]
        }, 
    "lrFactor": {
        "values": [ 0.2, 0.5]
    },                  
  
    "batch_size": {
        "values": [128]
    },
    "epochs": {
        "values": [100]
    }, 
    "NoLayers": {
        "values": [3, 4]
    },     
    
    "layer1": {
        "values": [2000, 3000, 4000]
    },
    "layer2": {
        "values": [1050, 2050]
    },     
    "layer4": {
        "values": [200, 400, 1050, 2050]
    },             
    "layer3": {
        "values": [200, 400, 1050, 2050]    }                
    }
}

In [ ]:
## Define training loop

In [ ]:
def train():
 
  wandb.init(tags=["SecondSweep3LayersBayesMethod"])  # defaults are over-ridden during the sweep
  config = wandb.config
#   AoModel = keras.Sequential([
#      keras.layers.InputLayer(19, name="digits"),
#      keras.layers.Dense(config.layer1, activation="relu"),
#      keras.layers.Dense(config.layer2, activation="relu"),
#      keras.layers.Dense(config.layer3, activation="relu"),
#      keras.layers.Dense(9, activation="linear", name="predictions"),

# ])
  inputs = keras.Input(shape=(19,))
  Layer1 = layers.Dense(config.layer1, activation="relu", name = "Layer1")
  x = Layer1(inputs)
  x = layers.Dense(config.layer2, activation="relu", name = "Layer2")(x)
  if(config.NoLayers >3):
      x = layers.Dense(config.layer3, activation="relu", name = "Layer3")(x)
  outputs = layers.Dense(9, name = "Output")(x)  
  x = layers.Dense(config.layer4, activation="relu", name = "Layer4")(x)

  AoModel = keras.Model(inputs=inputs, outputs=outputs, name="AOModel")

  #AoFunctionalModel.summary()

  earlyStopping = keras.callbacks.EarlyStopping(monitor = "val_loss", patience =10)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=config.lrFactor,
                                patience=5, min_lr=0.000001, verbose=1, cooldown= 5)

  opt = keras.optimizers.Adam(learning_rate=config.learning_rate)
  AoModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
  AoModel.fit(x_train_norm, y_train, batch_size=config.batch_size, epochs=config.epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr, earlyStopping, WandbCallback()])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])
  #AoModel.compile(loss= keras.losses.MeanSquaredError(), optimizer="adam", metrics= [tf.keras.metrics.RootMeanSquaredError()])

  #AoModel.fit(x_train_norm, y_train, batch_size=config.batch_size, epochs=config.epochs, verbose = 2,validation_split = 0.2 , callbacks=[WandbCallback()])
  #wandb.log({"learning_rate": "learning_rate"})

## Initialize sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity = "uwe-sterr",project="KerasAoSweep")
wandb.run

## Run sweep

In [ ]:
wandb.agent(sweep_id, train, count=400)